# Eat Safe ,Love 

# Notebook  Set up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(food_db.list_collection_names())


In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"{count} establishments have a hygiene score of 20")

# Display the first document in the results using pprint
result = establishments.find_one(query)
if result:
    pprint(result)
else:
    print("No establishments found with a hygiene score of 20")

In [ ]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(establishments.find(query))

# Display the number of rows in the DataFrame
print(f"There are {len(hygiene_df)} rows in the dataframe")

# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

# 2. Which establishments in London have a RatingValue greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query2 = {'LocalAuthorityName': {'$regex': 'London'}, 
          'RatingValue': {'$gte': '4'}
         }

# Use count_documents to display the number of documents in the result
rating_count = establishments.count_documents(query2)
print(f"{rating_count} establishments in London have a Rating Value >= 4")

# Display the first document in the results using pprint
pprint(establishments.find_one(query2))

In [ ]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find(query2))
rating_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
num_rows = len(rating_df)
print(f"There are {num_rows} rows in the dataframe")

# Display the first 10 rows of the DataFrame
rating_df.head(10)

# 3. What are the top 5 establishments with a RatingValue rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

query3 = {
    'RatingValue': '5',
    'geocode.latitude': {'$lte': (latitude + degree_search), '$gte': (latitude - degree_search)},
    'geocode.longitude': {'$lte': (longitude + degree_search), '$gte': (longitude - degree_search)}
}
sort = [('scores.Hygiene', 1)]

# Print the results
for result in establishments.find(query3).sort(sort).limit(5):
    pprint(result)

In [ ]:
# Convert result to Pandas DataFrame
location_df = pd.DataFrame(results)

# 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Define the pipeline stages
match_query = {'$match': {'scores.Hygiene': 0}}
group_query = {'$group': {'_id': '$LocalAuthorityName', 'count': {'$sum': 1}}}
sort_query = {'$sort': {'count': -1}}
pipeline = [match_query, group_query, sort_query]

# 3. Sorts the matches from highest to lowest
sort_query = {'$sort': {'count': -1}}
pipeline = [match_query, group_query, sort_query]

results = list(establishments.aggregate(pipeline))

# Display the number of documents in the result
num_documents = len(results)
print(f"There are {num_documents} documents")

# Display the first 10 results
pprint(results[:10])

In [ ]:
bad_hygiene_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
num_rows = len(bad_hygiene_df)
print(f"There are {num_rows} rows in the dataframe")

# Display the first 10 rows of the DataFrame
bad_hygiene_df.head(10)
